In [18]:
import re
import json
import pickle

import pandas as pd

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [39]:
with open("/Utilisateurs/umushtaq/am_reasoning/saved_models/pe_pipeline_prompt2_Qwen2.5-14B-Instruct-bnb-4bit/pe_pipeline_results_5.pickle", 'rb') as f:
    
    results = pickle.load(f)

In [40]:
grounds = results['grounds']
predictions = results['predictions']

In [41]:
grounds

['### Output:\n{"argument_types": [\'M\', \'M\', \'C\', \'C\', \'C\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\']}\n{"relation_types": [((8, 4), \'S\'), ((7, 4), \'S\'), ((6, 4), \'S\'), ((10, 11), \'S\'), ((9, 5), \'S\'), ((11, 5), \'S\')]}\n',
 '### Output:\n{"argument_types": [\'M\', \'M\', \'C\', \'C\', \'C\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'C\']}\n{"relation_types": [((6, 3), \'S\'), ((7, 4), \'A\'), ((8, 7), \'A\'), ((9, 4), \'S\'), ((10, 5), \'S\'), ((11, 5), \'S\')]}\n',
 '### Output:\n{"argument_types": [\'M\', \'M\', \'C\', \'C\', \'C\', \'C\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\', \'P\']}\n{"relation_types": [((7, 4), \'S\'), ((8, 9), \'S\'), ((10, 9), \'S\'), ((9, 4), \'S\'), ((11, 5), \'S\'), ((12, 5), \'S\'), ((13, 6), \'S\'), ((14, 6), \'S\'), ((15, 6), \'S\'), ((16, 6), \'S\'), ((17, 16), \'S\'), ((18, 16), \'S\'), ((19, 16), \'S\')]}\n',
 '### Output:\n{"argument_types": [\'M\', \'M\', \'C\', \'P\', \'P\', \'C\', 

In [4]:
def process_grounds(grounds):
    
    arg_types = []
    #rel_pairs = []
    rel_types = []
    
    for grounds in grounds:
        
        args = grounds.split("\n")[1].replace("'", '"')
        arg_types.append(json.loads(args)["argument_types"])
        
        rels = re.sub(r'\((\d+), (\d+)\)', r'[\1, \2]', grounds.split("\n")[2])
        rels = eval(rels)["relation_types"]
        rels = [(tuple(pair), label) for pair, label in rels]
        rel_types.append(rels)
        
    return arg_types, rel_types

In [5]:
grounds_acc, grounds_aric  = process_grounds(grounds)

In [6]:
def process_predictions(predictions):
    
    arg_types = []
    rel_types = []
    
    for pred in predictions:
        
        args = pred.split("\n")[1].replace("'", '"')
        arg_types.append(json.loads(args)["argument_types"])
        
        rels = re.sub(r'\((\d+), (\d+)\)', r'[\1, \2]', pred.split("\n")[2])
        rels = eval(rels)["relation_types"]
        rels = [(tuple(pair), label) for pair, label in rels]
        rel_types.append(rels)
        
    return arg_types, rel_types

In [7]:
predictions_acc, predictions_aric  = process_predictions(predictions)

In [8]:
for idx, (g, p) in enumerate(zip(grounds_acc, predictions_acc)):
    
    if len(p) > len(g):
        predictions_acc[idx] = p[:len(g)]

In [9]:
grounds = [elem for sublist in grounds_acc for elem in sublist]
predictions = [elem for sublist in predictions_acc for elem in sublist]

In [10]:
print(classification_report(grounds, predictions, digits=3))

              precision    recall  f1-score   support

           C      0.802     0.786     0.794       304
           M      0.973     0.935     0.953       153
           P      0.924     0.938     0.931       809

    accuracy                          0.901      1266
   macro avg      0.900     0.886     0.893      1266
weighted avg      0.901     0.901     0.901      1266



In [11]:
print(confusion_matrix(grounds, predictions))

[[239   4  61]
 [  9 143   1]
 [ 50   0 759]]


In [29]:
c_wrong = [idx for idx, elem in enumerate(grounds) if (elem == "C" and predictions[idx] == "P")]

In [30]:
len(c_wrong)

61

In [31]:
c_wrong

[27,
 48,
 62,
 121,
 135,
 160,
 167,
 168,
 180,
 195,
 250,
 276,
 278,
 304,
 343,
 363,
 410,
 415,
 437,
 442,
 486,
 509,
 523,
 532,
 534,
 545,
 594,
 624,
 705,
 730,
 736,
 766,
 779,
 797,
 815,
 844,
 909,
 915,
 940,
 941,
 970,
 974,
 987,
 992,
 1006,
 1038,
 1050,
 1058,
 1063,
 1075,
 1103,
 1105,
 1106,
 1177,
 1186,
 1204,
 1210,
 1227,
 1230,
 1233,
 1247]

In [42]:
df = pd.read_csv("/Utilisateurs/umushtaq/am_reasoning/data_files/pe_dataset_new.csv", index_col=0)

In [43]:
df

,ann_file_name,argument_ids,argument_types,arg_start_bounds,arg_end_bounds,argument_components,source_args,target_args,relation_types,essay_title,essay_text,split
0,essay001.ann,"['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8...","['MajorClaim', 'MajorClaim', 'Claim', 'Premise...","['503', '2154', '591', '716', '853', '1088', '...","['575', '2231', '714', '851', '1086', '1191', ...",['we should attach more importance to cooperat...,"['T4', 'T5', 'T6', 'T10', 'T9', 'T8']","['T3', 'T3', 'T3', 'T11', 'T11', 'T7']","['supports', 'supports', 'supports', 'supports...",Should students be taught to compete or to coo...,It is always said that competition can effecti...,TRAIN
1,essay002.ann,"['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8...","['MajorClaim', 'MajorClaim', 'Premise', 'Claim...","['391', '1936', '500', '1089', '626', '841', '...","['489', '2077', '624', '1156', '839', '946', '...",['they are able to sustain their cultural iden...,"['T9', 'T11', 'T12', 'T13', 'T5', 'T6', 'T7', ...","['T8', 'T10', 'T10', 'T10', 'T4', 'T4', 'T4', ...","['supports', 'supports', 'supports', 'supports...",More people are migrating to other countries t...,The last 50 years have seen an increasing numb...,TRAIN
2,essay003.ann,"['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8...","['MajorClaim', 'MajorClaim', 'Claim', 'Premise...","['352', '1568', '955', '487', '615', '786', '1...","['475', '1701', '1012', '613', '784', '943', '...",['it has contributed to the economic developme...,"['T10', 'T9', 'T6', 'T5', 'T4', 'T7']","['T8', 'T8', 'T3', 'T3', 'T3', 'T8']","['supports', 'supports', 'supports', 'supports...",International tourism is now more common than ...,The last decade has seen an increasing number ...,TRAIN
3,essay004.ann,"['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8...","['MajorClaim', 'MajorClaim', 'Claim', 'Claim',...","['262', '1663', '179', '953', '1578', '417', '...","['376', '1758', '239', '1031', '1624', '530', ...",['this industry has affected the cultural attr...,"['T8', 'T7', 'T6', 'T10', 'T9', 'T11']","['T4', 'T4', 'T4', 'T11', 'T5', 'T5']","['supports', 'supports', 'supports', 'supports...",International tourism is now more common than ...,The last 50 years have seen a significant incr...,TEST
4,essay005.ann,"['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8...","['MajorClaim', 'MajorClaim', 'Claim', 'Claim',...","['286', '1535', '388', '775', '1330', '566', '...","['359', '1652', '564', '883', '1425', '765', '...",['one who studies overseas will gain many skil...,"['T6', 'T7', 'T8', 'T9', 'T10', 'T11']","['T3', 'T4', 'T7', 'T4', 'T5', 'T5']","['supports', 'attacks', 'attacks', 'supports',...",Living and studying overseas,It is every student's desire to study at a goo...,TEST
...,...,...,...,...,...,...,...,...,...,...,...,...
397,essay398.ann,"['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8...","['Claim', 'MajorClaim', 'Premise', 'Premise', ...","['254', '1751', '409', '557', '647', '778', '9...","['318', '1951', '541', '645', '767', '897', '1...",['many different characters exist between male...,"['T3', 'T4', 'T5', 'T6', 'T7', 'T13', 'T14', '...","['T8', 'T8', 'T8', 'T8', 'T8', 'T12', 'T12', '...","['supports', 'supports', 'supports', 'supports...",We can not forcedly put the same numbers of ma...,There is a view that universities and colleges...,TEST
398,essay399.ann,"['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8...","['MajorClaim', 'MajorClaim', 'Claim', 'Premise...","['367', '1323', '527', '431', '661', '787', '9...","['429', '1390', '645', '525', '737', '951', '1...",['this is a worrying tread which has negative ...,"['T4', 'T5', 'T6', 'T8', 'T10']","['T3', 'T3', 'T7', 'T7', 'T9']","['supports', 'supports', 'supports', 'supports...","Drugs, alcohol and messy sex lives","Celebrities, for example movie stars, are usua...",TRAIN
399,essay400.ann,"['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8...","['MajorClaim', 'MajorClaim', 'Claim', 'Premise...","['274', '1619', '736', '541', '805', '389', '1...","['387', '1812', '796', '723',

In [44]:
df_test = df[df.split == "TEST"].reset_index(drop=True)

In [45]:
df_test

,ann_file_name,argument_ids,argument_types,arg_start_bounds,arg_end_bounds,argument_components,source_args,target_args,relation_types,essay_title,essay_text,split
0,essay004.ann,"['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8...","['MajorClaim', 'MajorClaim', 'Claim', 'Claim',...","['262', '1663', '179', '953', '1578', '417', '...","['376', '1758', '239', '1031', '1624', '530', ...",['this industry has affected the cultural attr...,"['T8', 'T7', 'T6', 'T10', 'T9', 'T11']","['T4', 'T4', 'T4', 'T11', 'T5', 'T5']","['supports', 'supports', 'supports', 'supports...",International tourism is now more common than ...,The last 50 years have seen a significant incr...,TEST
1,essay005.ann,"['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8...","['MajorClaim', 'MajorClaim', 'Claim', 'Claim',...","['286', '1535', '388', '775', '1330', '566', '...","['359', '1652', '564', '883', '1425', '765', '...",['one who studies overseas will gain many skil...,"['T6', 'T7', 'T8', 'T9', 'T10', 'T11']","['T3', 'T4', 'T7', 'T4', 'T5', 'T5']","['supports', 'attacks', 'attacks', 'supports',...",Living and studying overseas,It is every student's desire to study at a goo...,TEST
2,essay006.ann,"['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8...","['MajorClaim', 'MajorClaim', 'Claim', 'Claim',...","['174', '1667', '211', '309', '779', '1092', '...","['209', '1759', '293', '407', '892', '1147', '...","['studying abroad has many advantages', 'study...","['T7', 'T8', 'T10', 'T9', 'T11', 'T12', 'T13',...","['T4', 'T9', 'T9', 'T4', 'T5', 'T5', 'T6', 'T6...","['supports', 'supports', 'supports', 'supports...",Studies abroad and the cultural aspect of the ...,Studying abroad is one very common thing that ...,TEST
3,essay021.ann,"['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8...","['MajorClaim', 'MajorClaim', 'Claim', 'Premise...","['325', '1701', '118', '218', '250', '479', '1...","['384', '1769', '207', '248', '308', '569', '1...",['advertising is the major reason for high sal...,"['T5', 'T4', 'T9', 'T11', 'T12', 'T10', 'T13',...","['T3', 'T3', 'T6', 'T13', 'T13', 'T6', 'T6', '...","['supports', 'supports', 'supports', 'supports...",Advertisements affects on consumer goods,"Everyday, companies offer a huge number of dif...",TEST
4,essay042.ann,"['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8...","['MajorClaim', 'MajorClaim', 'Claim', 'Claim',...","['1562', '1665', '1776', '346', '1055', '425',...","['1648', '1769', '1856', '423', '1203', '533',...",['a higher pay can be one of the incentive if ...,"['T6', 'T7', 'T8', 'T9', 'T10', 'T11']","['T4', 'T4', 'T7', 'T7', 'T5', 'T5']","['supports', 'supports', 'supports', 'supports...",Paying more money is the only motivation to ma...,The purpose of this essay is to discuss whethe...,TEST
...,...,...,...,...,...,...,...,...,...,...,...,...
75,essay373.ann,"['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8...","['MajorClaim', 'Claim', 'Premise', 'Premise', ...","['607', '1063', '790', '950', '719', '1229', '...","['684', '1146', '933', '986', '788', '1295', '...",['it is no evidence about the reduction of cri...,"['T4', 'T22', 'T3', 'T5', 'T14', 'T12', 'T13',...","['T22', 'T2', 'T2', 'T2', 'T15', 'T15', 'T15',...","['supports', 'supports', 'supports', 'supports...",Capital punishment; 51% countries have polishe...,"""Capital punishment or the death penalty is a ...",TEST
76,essay382.ann,"['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8...","['MajorClaim', 'Claim', 'MajorClaim', 'Claim',...","['274', '1429', '1369', '409', '534', '657', '...","['386', '1502', '1424', '532', '655', '800', '...","[""the richness as well as convenience of techn...","['T5', 'T6', 'T8', 'T7', 'T9']","['T4', 'T4', 'T7', 'T10', 'T10']","['supports', 'supports', 'supports', 'supports...",Technology helps student learn more informatio...,"Nowadays, technology plays an indispensable ro...",TEST
77,essay386.ann,"['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8...","['MajorClaim', 'MajorClaim', 'Claim', 'Premise...","['324', '1574', '1629', '518', '748', '433', '...

In [65]:
df_expanded = df_test.explode('argument_components').reset_index(drop=True)

In [66]:
df_expanded

,ann_file_name,argument_ids,argument_types,arg_start_bounds,arg_end_bounds,argument_components,source_args,target_args,relation_types,essay_title,essay_text,split
0,essay004.ann,"['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8...","['MajorClaim', 'MajorClaim', 'Claim', 'Claim',...","['262', '1663', '179', '953', '1578', '417', '...","['376', '1758', '239', '1031', '1624', '530', ...",['this industry has affected the cultural attr...,"['T8', 'T7', 'T6', 'T10', 'T9', 'T11']","['T4', 'T4', 'T4', 'T11', 'T5', 'T5']","['supports', 'supports', 'supports', 'supports...",International tourism is now more common than ...,The last 50 years have seen a significant incr...,TEST
1,essay005.ann,"['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8...","['MajorClaim', 'MajorClaim', 'Claim', 'Claim',...","['286', '1535', '388', '775', '1330', '566', '...","['359', '1652', '564', '883', '1425', '765', '...",['one who studies overseas will gain many skil...,"['T6', 'T7', 'T8', 'T9', 'T10', 'T11']","['T3', 'T4', 'T7', 'T4', 'T5', 'T5']","['supports', 'attacks', 'attacks', 'supports',...",Living and studying overseas,It is every student's desire to study at a goo...,TEST
2,essay006.ann,"['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8...","['MajorClaim', 'MajorClaim', 'Claim', 'Claim',...","['174', '1667', '211', '309', '779', '1092', '...","['209', '1759', '293', '407', '892', '1147', '...","['studying abroad has many advantages', 'study...","['T7', 'T8', 'T10', 'T9', 'T11', 'T12', 'T13',...","['T4', 'T9', 'T9', 'T4', 'T5', 'T5', 'T6', 'T6...","['supports', 'supports', 'supports', 'supports...",Studies abroad and the cultural aspect of the ...,Studying abroad is one very common thing that ...,TEST
3,essay021.ann,"['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8...","['MajorClaim', 'MajorClaim', 'Claim', 'Premise...","['325', '1701', '118', '218', '250', '479', '1...","['384', '1769', '207', '248', '308', '569', '1...",['advertising is the major reason for high sal...,"['T5', 'T4', 'T9', 'T11', 'T12', 'T10', 'T13',...","['T3', 'T3', 'T6', 'T13', 'T13', 'T6', 'T6', '...","['supports', 'supports', 'supports', 'supports...",Advertisements affects on consumer goods,"Everyday, companies offer a huge number of dif...",TEST
4,essay042.ann,"['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8...","['MajorClaim', 'MajorClaim', 'Claim', 'Claim',...","['1562', '1665', '1776', '346', '1055', '425',...","['1648', '1769', '1856', '423', '1203', '533',...",['a higher pay can be one of the incentive if ...,"['T6', 'T7', 'T8', 'T9', 'T10', 'T11']","['T4', 'T4', 'T7', 'T7', 'T5', 'T5']","['supports', 'supports', 'supports', 'supports...",Paying more money is the only motivation to ma...,The purpose of this essay is to discuss whethe...,TEST
...,...,...,...,...,...,...,...,...,...,...,...,...
75,essay373.ann,"['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8...","['MajorClaim', 'Claim', 'Premise', 'Premise', ...","['607', '1063', '790', '950', '719', '1229', '...","['684', '1146', '933', '986', '788', '1295', '...",['it is no evidence about the reduction of cri...,"['T4', 'T22', 'T3', 'T5', 'T14', 'T12', 'T13',...","['T22', 'T2', 'T2', 'T2', 'T15', 'T15', 'T15',...","['supports', 'supports', 'supports', 'supports...",Capital punishment; 51% countries have polishe...,"""Capital punishment or the death penalty is a ...",TEST
76,essay382.ann,"['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8...","['MajorClaim', 'Claim', 'MajorClaim', 'Claim',...","['274', '1429', '1369', '409', '534', '657', '...","['386', '1502', '1424', '532', '655', '800', '...","[""the richness as well as convenience of techn...","['T5', 'T6', 'T8', 'T7', 'T9']","['T4', 'T4', 'T7', 'T10', 'T10']","['supports', 'supports', 'supports', 'supports...",Technology helps student learn more informatio...,"Nowadays, technology plays an indispensable ro...",TEST
77,essay386.ann,"['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8...","['MajorClaim', 'MajorClaim', 'Claim', 'Premise...","['324', '1574', '1629', '518', '748', '433', '...

In [77]:
arg_ids = df_test.argument_ids.tolist()
arg_comps = df_test.argument_components.tolist()
arg_types = df_test.argument_types.tolist()

In [78]:
import ast

In [82]:
len(arg_ids), len(arg_comps), len(arg_types)

(80, 1266, 1266)

In [80]:
arg_comps = [ast.literal_eval(elem) for elem in arg_comps]
arg_types = [ast.literal_eval(elem) for elem in arg_types]

In [81]:
arg_comps = [item for sublist in arg_comps for item in sublist]
arg_types = [item for sublist in arg_types for item in sublist]

In [87]:
args = {}

for idx, (i, j) in enumerate(zip(arg_comps, arg_types)):
        
    args[idx] = i + "|" + j

In [88]:
len(args)

1266

In [89]:
args

{0: 'this industry has affected the cultural attributes and damaged the natural environment of the tourist destinations|MajorClaim',
 1: 'the tourism has created threatening pressure on both the sociocultural and natural environments|MajorClaim',
 2: 'the tourism bring large profit for the destination countries|Claim',
 3: 'international tourism can create negative impacts on the destination countries|Claim',
 4: 'tourism has threatened the nature environments|Claim',
 5: 'tourists from different cultures will probably cause changes to the cultural identity of the tourist destinations|Premise',
 6: 'Take Thailand for example, in the Vietnam War, many American soldiers came to Thailand for a break and involved in sexual and drug activities, these huge demands caused many local businesses opened and expanded, even illegally involved in under-age prostitutes to maximize their profits|Premise',
 7: 'This was due to the lack of adequate controls by authorities and lead to a bad image of Tha

In [91]:
df_x = pd.DataFrame.from_dict(args, orient="index")

In [92]:
df_x

,0
0,this industry has affected the cultural attrib...
1,the tourism has created threatening pressure o...
2,the tourism bring large profit for the destina...
3,international tourism can create negative impa...
4,tourism has threatened the nature environments...
...,...
1261,universities should encourage more girls to ch...
1262,this could avoid imbalance of gender in some s...
1263,It would affect students' mental health to stu...
1264,she is unlikely to focus on her subject|Premise


In [94]:
df_x.columns

RangeIndex(start=0, stop=1, step=1)

In [95]:
df_split = df_x[df_x.columns[0]].str.split('|', expand=True)

In [96]:
df_split

,0,1
0,this industry has affected the cultural attrib...,MajorClaim
1,the tourism has created threatening pressure o...,MajorClaim
2,the tourism bring large profit for the destina...,Claim
3,international tourism can create negative impa...,Claim
4,tourism has threatened the nature environments,Claim
...,...,...
1261,universities should encourage more girls to ch...,Claim
1262,this could avoid imbalance of gender in some s...,Premise
1263,It would affect students' mental health to stu...,Premise
1264,she is unlikely to focus on her subject,Premise


In [97]:
selected_rows = df_split.loc[c_wrong].reset_index(drop=True)

In [98]:
selected_rows

,0,1
0,the new academic experience that the students ...,Claim
1,Advertisement is the most effective way to cre...,Claim
2,they will be feeling more cared for and theref...,Claim
3,the advertising expenses lead to a higher prod...,Claim
4,"rigorous exercise can also hone one's will, wh...",Claim
...,...,...
56,"with some of serious criminals, capital punish...",Claim
57,Another point that students may benefit from t...,Claim
58,they provide tremendous incentive and horizon ...,Claim
59,the classmates themselves creat a competitive ...,Claim


In [99]:
df_z = pd.read_csv("/Utilisateurs/umushtaq/am_reasoning/data_files/pe_dataset.csv", index_col=0)

In [100]:
df_z

,split,title,argument_bound_1,argument_bound_2,argument_id,ac,label,essay
0,TRAIN,Should students be taught to compete or to coo...,503,575,1,we should attach more importance to cooperatio...,MajorClaim,Should students be taught to compete or to coo...
1,TRAIN,Should students be taught to compete or to coo...,591,714,2,"through cooperation, children can learn about ...",Claim,Should students be taught to compete or to coo...
2,TRAIN,Should students be taught to compete or to coo...,716,851,3,What we acquired from team work is not only ho...,Premise,Should students be taught to compete or to coo...
3,TRAIN,Should students be taught to compete or to coo...,853,1086,4,"During the process of cooperation, children ca...",Premise,Should students be taught to compete or to coo...
4,TRAIN,Should students be taught to compete or to coo...,1088,1191,5,All of these skills help them to get on well w...,Premise,Should students be taught to compete or to coo...
...,...,...,...,...,...,...,...,...
6084,TRAIN,Children should studying hard or playing sport...,1275,1339,11,indirectly they will learn how to socialize ea...,Premise,Children should studying hard or playing sport...
6085,TRAIN,Children should studying hard or playing sport...,1341,1388,12,That will make children getting lots of friends,Premise,Children should studying hard or playing sport...
6086,TRAIN,Children should studying hard or playing sport...,1393,1436,13,they can contribute positively to community,Premise,Children should studying hard or playing sport...
6087,TRAIN,Children should studying hard or playing sport...,1448,1525,14,playing sport makes children getting healthy a...,Premise,Children should studying hard or playing sport...


In [101]:
df_zz = df_z[df_z.split == "TEST"].reset_index(drop=True)

In [102]:
df_zz

,split,title,argument_bound_1,argument_bound_2,argument_id,ac,label,essay
0,TEST,International tourism is now more common than ...,179,239,1,the tourism bring large profit for the destina...,Claim,International tourism is now more common than ...
1,TEST,International tourism is now more common than ...,262,376,2,this industry has affected the cultural attrib...,MajorClaim,International tourism is now more common than ...
2,TEST,International tourism is now more common than ...,417,530,3,tourists from different cultures will probably...,Premise,International tourism is now more common than ...
3,TEST,International tourism is now more common than ...,532,818,4,"Take Thailand for example, in the Vietnam War,...",Premise,International tourism is now more common than ...
4,TEST,International tourism is now more common than ...,820,924,5,This was due to the lack of adequate controls ...,Premise,International tourism is now more common than ...
...,...,...,...,...,...,...,...,...
1261,TEST,We can not forcedly put the same numbers of ma...,1394,1463,12,this also can block the girl's future developm...,Premise,We can not forcedly put the same numbers of ma...
1262,TEST,We can not forcedly put the same numbers of ma...,1484,1589,13,universities should encourage more girls to ch...,Claim,We can not forcedly put the same numbers of ma...
1263,TEST,We can not forcedly put the same numbers of ma...,1595,1648,14,this could avoid imbalance of gender in some s...,Premise,We can not forcedly put the same numbers of ma...
1264,TEST,We can not forcedly put the same numbers of ma...,1650,1734,15,It would affect students' mental health to stu...,Premise,We can not forcedly put the same numbers of ma...


In [106]:
selected_rows.columns = ["ac", "label"]

In [107]:
selected_rows

,ac,label
0,the new academic experience that the students ...,Claim
1,Advertisement is the most effective way to cre...,Claim
2,they will be feeling more cared for and theref...,Claim
3,the advertising expenses lead to a higher prod...,Claim
4,"rigorous exercise can also hone one's will, wh...",Claim
...,...,...
56,"with some of serious criminals, capital punish...",Claim
57,Another point that students may benefit from t...,Claim
58,they provide tremendous incentive and horizon ...,Claim
59,the classmates themselves creat a competitive ...,Claim


In [110]:
merged_df = selected_rows.merge(df_zz, on="ac", how="inner").reset_index(drop=True)

In [111]:
merged_df

,ac,label_x,split,title,argument_bound_1,argument_bound_2,argument_id,label_y,essay
0,the new academic experience that the students ...,Claim,TEST,Studies abroad and the cultural aspect of the ...,779,892,8,Claim,Studies abroad and the cultural aspect of the ...
1,Advertisement is the most effective way to cre...,Claim,TEST,Advertisements affects on consumer goods,1097,1167,12,Claim,Advertisements affects on consumer goods\n\nEv...
2,they will be feeling more cared for and theref...,Claim,TEST,Paying more money is the only motivation to ma...,1776,1856,11,Claim,Paying more money is the only motivation to ma...
3,the advertising expenses lead to a higher prod...,Claim,TEST,Should all the advertisements be banned?,1221,1382,7,Claim,Should all the advertisements be banned?\n\nNe...
4,"rigorous exercise can also hone one's will, wh...",Claim,TEST,The best way to relax is by exercises,1386,1507,11,Claim,The best way to relax is by exercises\n\nIn mo...
...,...,...,...,...,...,...,...,...,...
56,"with some of serious criminals, capital punish...",Claim,TEST,Capital punishment; 51% countries have polishe...,1607,1676,13,Claim,Capital punishment; 51% countries have polishe...
57,Another point that students may benefit from t...,Claim,TEST,Technology helps student learn more informatio...,802,892,5,Claim,Technology helps student learn more informatio...
58,they provide tremendous incentive and horizon ...,Claim,TEST,Classmates' impacts on children's performance ...,1629,1756,10,Claim,Classmates' impacts on children's performance ...
59,the classmates themselves creat a competitive ...,Claim,TEST,Classmates' impacts on children's performance ...,433,516,2,Claim,Classmates' impacts on children's performance ...


In [113]:
len(merged_df.essay.unique().tolist())

44

In [114]:
len(merged_df.title.unique().tolist())

44